In [1]:
import pandas as pd
import vectorbtpro as vbt
from datetime import datetime, timedelta
import pytz
import numpy as np
import pandas_ta as ta
import logging
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
import multiprocessing as mp
mp.set_start_method('fork')


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:

start = "2018-01-01"
# Enter your parameters here
metric = 'total_return'

start_date = datetime(2020, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)

# The following is the number of days to look back for the analysis
time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
freq = '1h'

vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
# vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
# vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# binance_data.save("binance_data.pkl")

# If you already have the data downloaded, you can load it
# binance_data = vbt.BinanceData.load("binance_data.pkl")
# binance_data = binance_data.update() if you want to update it.
futures_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl'
# You can grab the necessary file from this google drive link. https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link

If you don't have the datafile yet you can grab it from this [link](https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link)

In [3]:
btc = vbt.BinanceData.load(futures_path)
df = btc.get()

In [4]:
# create a signal array the same length as the dataframe
signal = np.zeros(len(df))
signal = pd.Series(signal, index=df.index)
# set the signal to 1 at midnight UTC
signal[df.at_time('00:00').index] = 1
# Create a -1 right before the signal
signal[df.at_time('12:00').index] = -1 # random short signals at noon
df['signal'] = signal
# Convert to int
df['signal'] = df['signal'].astype(int)
df['signal'].sum()

0

In [6]:



class LONG_SHORT_Underwater_w_decay_and_deleverage(Strategy):
    # All of the following variables can be used during optimization
    initial_position_size = 0.3
    percent_invested_threshold = 0.50
    atr_length = 14  # 14 days
    atr_multiplier = 0.5
    add_size = 0.30
    delay_period = 1000
    delta_time = 5000
    upper_bound_profit_target = 0.10
    lower_bound_loss_threshold = -0.05
    take_profit_loss_reduction = (
        -0.1
    )  # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = 0.30  # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim

    def SIGNAL(self):
        return self.data.signal

    def ATR(self, df, length):
        return df.ta.atr(length=length)

    def bars_since_first_trade(self, use_for_indexing=False):
        """
        Calculate the number of bars since the first trade was entered.
        If use for indexing is true then it will return 1 if there are no trades.
        This way you can retrieve the last element of a list by applying a - to the return value.
        eg. self.equity[-self.bars_since_first_trade(use_for_indexing=True):] gives you the array of your account's equity since you entered the first trade that is currently open
        """
        if len(self.trades) > 0:
            self.first_trade_entry_bar = self.trades[0].entry_bar
            bars_since_first_trade = len(self.data.Close) - self.first_trade_entry_bar
            return bars_since_first_trade
        elif use_for_indexing:
            return 1
        else:
            return 0

    def custom_decay_func(
        self,
        x,
        delay_period,
        upper_bound_profit_target,
        lower_bound_loss_threshold,
        delta_time,
    ):
        """
        This function is used to calculate the decayed take profit x represents the number of bars since the first trade was entered
        x is the number of bars since the first trade was entered
        delay_period is the number of bars to wait before starting to decay
        upper_bound_profit_target is the upper bound of the take profit
        lower_bound_loss_threshold is the lower bound of the take profit
        delta_time is the number of bars over which to transition from the upper bound to the lower bound
        """
        if x <= delay_period:
            return upper_bound_profit_target
        elif delay_period < x < delay_period + delta_time:
            # Calculate the x value for the cos function
            transition_x = (x - delay_period) / delta_time * np.pi
            # Calculate the decayed take profit
            return (-np.cos(transition_x) + 1) / 2 * (
                lower_bound_loss_threshold - upper_bound_profit_target
            ) + upper_bound_profit_target
        else:
            return lower_bound_loss_threshold

    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
        self.atr = self.I(self.ATR, self.data.df, self.atr_length)
        self.daily_atr = resample_apply("1D", self.ATR, self.data.df, length=14)
        count_NaN = len(self.atr) - len(
            pd.Series(self.atr).dropna()
        )  # This is used for the progress bar
        self.length_of_data = (
            len(self.data.Close) - count_NaN
        )  # This is used for the progress bar
        self.equity_during_trade = []  # Keeps a list for the equity during the trade
        self.long_short_flag = (
            None  # This is used to keep track of whether we are long or short
        )

    def next(self):
        super().next()

        price = self.data.Close[-1]
        position_value = self.position.size * price
        percent_invested = (
            position_value / self.equity
        )  # this will come in handy if we decide to change behavior once XX% is invested
        atr_threshold_pct = (
            self.atr_multiplier * self.daily_atr[-1] / price
        )  # This is the ATR threshold times a multiplier calculated as a percentage of price
        bars_since_trade_open = self.bars_since_first_trade(use_for_indexing=True)
        highly_leveraged = percent_invested > self.percent_invested_threshold
        # Calculate the decayed take profit
        decayed_take_profit = self.custom_decay_func(
            bars_since_trade_open,
            self.delay_period,
            self.upper_bound_profit_target,
            self.lower_bound_loss_threshold,
            self.delta_time,
        )

        # Keep a running list of the self.equity values while we have a trade open
        if self.position:  # TODO work on this for long versus short
            self.equity_during_trade.append(self.equity)
        else:
            self.equity_during_trade = []

        low_point_in_trade = min(self.equity_during_trade, default=0)
        bounce_from_low_pct = (lambda x: (self.equity - x) / x if x != 0 else 0)(
            low_point_in_trade
        )  # The lambda function is used to avoid a divide by zero error

        # Opening a Trade on a signal from the LSTM
        if not self.position:
            if self.signal == 1:
                self.buy(size=self.initial_position_size)
                self.long_short_flag = 1
            elif self.signal == -1:
                self.sell(size=self.initial_position_size)
                self.long_short_flag = -1

        # If we are in a trade and it meets our profit criteria then close the trade
        elif self.position.pl_pct > decayed_take_profit:
            self.position.close()

        # Deleverage if we are over a certain percent invested and the market is recovering from the low point
        elif (
            highly_leveraged and bounce_from_low_pct > 1.5 * atr_threshold_pct
        ):  # TODO: maybe make this a parameter we can tune
            # TODO: this is crude but it works for now
            if (
                self.position.pl_pct
                > decayed_take_profit + self.take_profit_loss_reduction
            ):
                self.position.close(portion=self.deleverage_pct)

        # If we are in a trade and it meets our loss criteria then close a portion of the trade on a bounce from the low point
        elif self.position and self.position.pl_pct < -atr_threshold_pct:
            # Check to see if we are also down on our last trade
            if self.trades[-1].pl_pct < -atr_threshold_pct:
                if self.long_short_flag == 1:
                    self.buy(size=self.add_size)
                elif self.long_short_flag == -1:
                    self.sell(size=self.add_size)


start = "2022-05-01"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2022-12-03"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    df.loc[start:end],
    LONG_SHORT_Underwater_w_decay_and_deleverage,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=0.5,
)
stats = bt.run()
print(stats)
bt.plot(resample=False)

Start                     2022-05-01 00:00...
End                       2022-12-03 23:59...
Duration                    216 days 23:59:00
Exposure Time [%]                   85.945661
Equity Final [$]                   99492960.5
Equity Peak [$]                   113950032.8
Return [%]                          -0.507039
Buy & Hold Return [%]                -55.1497
Return (Ann.) [%]                   -0.851379
Volatility (Ann.) [%]               62.981783
Sharpe Ratio                        -0.013518
Sortino Ratio                       -0.018766
Calmar Ratio                        -0.019494
Max. Drawdown [%]                  -43.674546
Avg. Drawdown [%]                   -0.893446
Max. Drawdown Duration      180 days 09:23:00
Avg. Drawdown Duration        2 days 02:16:00
# Trades                                  416
Win Rate [%]                        42.067308
Best Trade [%]                      11.887688
Worst Trade [%]                    -34.566849
Avg. Trade [%]                    

In [7]:
bt.plot(resample=False)

GridPlot(id='p1721', ...)